In [1]:
# pip install -r requirements.txt --quiet

In [2]:
import pandas as pd
import data.building_ingestion_utils as biu
# import modeling_utils as mu
import data.data_utils as du
# import optimization_utils as ou
# import weather_ingestion_utils as wiu
import data.energy_price_utils as epu
import json

directory = 'data/site_data/industrial_sites'

dataframes = du.restructure_dataframes(biu.get_dataframes(directory, ';'))

In [3]:
print(f"Total dataframes created: {len(dataframes)}")

for name, df in dataframes.items():
    print(f'DataFrame: {name} loaded with shape: {df.shape}')

Total dataframes created: 3
DataFrame: industrial_site2 loaded with shape: (42760, 29)
DataFrame: industrial_site3 loaded with shape: (42760, 44)
DataFrame: industrial_site1 loaded with shape: (42760, 54)


In [4]:
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['DataFrame Name', 'Time', '01 General_Electric_Active Energy', '02 Production_Electric_Active Energy', '03 Chiller Group_Electric_Active Energy', '04 UTA_Electric_Active Energy', '05 Compressors_Electric_Active Energy', '06 Offices_Electric_Active Energy', '07 Data Center_Electric_Active Energy', '08 Technological Centers_Electric_Active Energy', 'General_Electric_Active Energy', '01 General_Electric_Power Factor', '02 Production_Electric_Power Factor', '03 Chiller Group_Electric_Power Factor', '04 UTA_Electric_Power Factor', '05 Compressors_Electric_Power Factor', '06 Offices_Electric_Power Factor', '07 Data Center_Electric_Power Factor', '08 Technological Centers_Electric_Power Factor', 'General_Electric_Power Factor', '01 General_Electric_Active Power', '02 Production_Electric_Active Power', '03 Chiller Group_Electric_Active Power', '04 UTA_Electric_Active Power', '05 Compressors_Electric_Active Power', '06 Offices_Electric_Active Power', '07 Da

Data is saved in dictionary of dataframes called `dataframe`

In [5]:
#   new_entry_key = 'energy'
#   new_entry_value = 'kWh'
# add_entry_to_units_dict(file_path, new_entry_key, new_entry_value, verbose=False)

units_dict = biu.read_units_dict_from_json('data/unit_dict.json')

dataframes = biu.add_units_to_column_names(dataframes, units_dict)


du.column_presence_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Technical_Efficiency (%),General_Technical_Flow Rate (m3/h),General_Technical_Pressure (bar),General_Technical_Temperature (C),General_Vapour_Flow Rate (m3/h),General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time
0,industrial_site2,Column not present,Column not present,Column not present,float64,float64,float64,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,object
1,industrial_site3,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,float64,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,object
2,industrial_site1,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,...,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,float64,float64,object


In [6]:
du.nan_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Technical_Efficiency (%),General_Technical_Flow Rate (m3/h),General_Technical_Pressure (bar),General_Technical_Temperature (C),General_Vapour_Flow Rate (m3/h),General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time
0,No NaNs,Column not present,Column not present,Column not present,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,No NaNs
1,No NaNs,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,No NaNs
2,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Some NaNs,Some NaNs,Some NaNs,...,Column not present,Column not present,Column not present,Column not present,Some NaNs,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs


In [7]:
datetime_format = '%d/%m/%y %H:%M'
dataframes = biu.convert_and_transform_date_columns(dict_of_dfs=dataframes, datetime_format=datetime_format, date_column='Time', verbose=True)
du.column_presence_checker(dataframes)



DataFrame: industrial_site2 - Converted Time to datetime
DataFrame: industrial_site3 - Converted Time to datetime
DataFrame: industrial_site1 - Converted Time to datetime


,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time,day_of_week_cos,day_of_week_sin,is_weekend,month_cos,month_sin
0,industrial_site2,Column not present,Column not present,Column not present,float64,float64,float64,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,datetime64[ns],float64,float64,int64,float64,float64
1,industrial_site3,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,datetime64[ns],float64,float64,int64,float64,float64
2,industrial_site1,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,...,float64,float64,float64,float64,datetime64[ns],float64,float64,int64,float64,float64


In [8]:
du.nan_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time,day_of_week_cos,day_of_week_sin,is_weekend,month_cos,month_sin
0,No NaNs,Column not present,Column not present,Column not present,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
1,No NaNs,Some NaNs,Some NaNs,Some NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
2,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Some NaNs,Some NaNs,Some NaNs,...,Some NaNs,Some NaNs,Some NaNs,Some NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs


In [9]:
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,DataFrame Name,Time,01 General_Electric_Active Energy (kWh),02 Production_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Energy (kWh),04 UTA_Electric_Active Energy (kWh),05 Compressors_Electric_Active Energy (kWh),06 Offices_Electric_Active Energy (kWh),07 Data Center_Electric_Active Energy (kWh),08 Technological Centers_Electric_Active Energy (kWh),...,05 Compressors_Electric_Active Power (kW),06 Offices_Electric_Active Power (kW),07 Data Center_Electric_Active Power (kW),08 Technological Centers_Electric_Active Power (kW),General_Electric_Active Power (kW),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,industrial_site2,2022-07-01 00:00:00,133.0,16.0,52.0,10.0,4.0,1.0,0.0,7.0,...,16.7,2.94,0.26,34.7,75.6,0,-0.5,-0.866025,-0.433884,-0.900969
1,industrial_site2,2022-07-01 00:15:00,131.0,16.0,51.0,10.0,4.0,0.0,0.0,8.0,...,15.5,3.02,0.26,30.5,76.4,0,-0.5,-0.866025,-0.433884,-0.900969
2,industrial_site2,2022-07-01 00:30:00,129.0,15.0,51.0,10.0,5.0,1.0,0.0,7.0,...,15.7,2.70,0.25,29.5,76.9,0,-0.5,-0.866025,-0.433884,-0.900969
3,industrial_site2,2022-07-01 00:45:00,123.0,13.0,46.0,10.0,4.0,1.0,0.0,7.0,...,16.9,2.82,0.26,28.8,74.6,0,-0.5,-0.866025,-0.433884,-0.900969
4,industrial_site2,2022-07-01 01:00:00,128.0,17.0,47.0,10.0,4.0,1.0,0.0,8.0,...,18.1,3.24,0.26,30.5,74.7,0,-0.5,-0.866025,-0.433884,-0.900969




Preview of DataFrame for industrial_site3:


,DataFrame Name,Time,01 General Transformer 1234_Electric_Active Energy (kWh),02 Chiller Group_Electric_Active Energy (kWh),03 Aspirator_Electric_Active Energy (kWh),04 Compressed Air_Electric_Active Energy (kWh),05 Weaving_Electric_Active Energy (kWh),06 Ironing_Electric_Active Energy (kWh),07 UPS_Electric_Active Energy (kWh),08 General Services_Electric_Active Energy (kWh),...,10 Warehouses_Electric_Active Power (kW),11 Winding_Electric_Active Power (kW),12 Others_Electric_Active Power (kW),General_Technical_Active Power (kW),General_Technical_Flow Rate (m3/h),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,industrial_site3,2022-07-01 00:00:00,579.0,82.6,103.0,144.0,98.5,73.7,0.536,8.37,...,10.8,3.63,491.0,0.080,750.0,0,-0.5,-0.866025,-0.433884,-0.900969
1,industrial_site3,2022-07-01 00:15:00,513.0,80.5,103.0,132.0,97.8,49.9,0.552,7.71,...,10.6,3.02,476.0,0.144,928.0,0,-0.5,-0.866025,-0.433884,-0.900969
2,industrial_site3,2022-07-01 00:30:00,505.0,90.1,118.0,138.0,107.0,47.6,0.484,8.60,...,10.6,2.96,482.0,-0.036,588.0,0,-0.5,-0.866025,-0.433884,-0.900969
3,industrial_site3,2022-07-01 00:45:00,512.0,85.0,103.0,131.0,98.2,45.5,0.484,7.84,...,10.6,7.00,477.0,-0.140,638.0,0,-0.5,-0.866025,-0.433884,-0.900969
4,industrial_site3,2022-07-01 01:00:00,510.0,84.6,103.0,132.0,97.8,45.5,0.488,7.78,...,10.6,6.77,470.0,-1.200,844.0,0,-0.5,-0.866025,-0.433884,-0.900969




Preview of DataFrame for industrial_site1:


,DataFrame Name,Time,01 Normal General_Electric_Active Energy (kWh),02 Chiller_Electric_Active Energy (kWh),03 Dyeing_Electric_Active Energy (kWh),04 Ironing_Electric_Active Energy (kWh),05 Purifier_Electric_Active Energy (kWh),07 Technological Centers_Electric_Active Energy (kWh),08 Offices Changing Rooms_Electric_Active Energy (kWh),09 Compressed Air_Electric_Active Energy (kWh),...,17 Print_Electric_Active Power (kW),General_Electric_Active Power (kW),General_Natural Gas_Flow Rate (m3/h),General_Vapour_Flow Rate (m3/h),General_Water_Flow Rate (m3/h),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,industrial_site1,2022-07-01 00:00:00,441.0,60.5,60.4,47.4,27.9,32.7,14.6,37.6,...,69.3,471.0,253.00,8813.0,766.0,0,-0.5,-0.866025,-0.433884,-0.900969
1,industrial_site1,2022-07-01 00:15:00,412.0,60.7,57.2,46.2,22.5,27.7,17.8,40.7,...,106.0,480.0,0.09,8636.0,404.0,0,-0.5,-0.866025,-0.433884,-0.900969
2,industrial_site1,2022-07-01 00:30:00,440.0,56.4,62.4,40.3,24.0,30.6,17.7,41.6,...,104.0,519.0,864.00,16272.0,784.0,0,-0.5,-0.866025,-0.433884,-0.900969
3,industrial_site1,2022-07-01 00:45:00,448.0,54.4,60.0,48.6,24.7,30.7,17.8,40.1,...,109.0,510.0,275.00,11009.0,803.0,0,-0.5,-0.866025,-0.433884,-0.900969
4,industrial_site1,2022-07-01 01:00:00,410.0,53.8,59.5,47.7,24.7,31.5,17.5,41.4,...,111.0,520.0,417.00,10018.0,665.0,0,-0.5,-0.866025,-0.433884,-0.900969


# Create Target Column

## Create Total Consumption (kW) Column

In [10]:
# # Assuming you have already loaded the dataframes into the `dataframes` dictionary
# target_columns = ['Active Energy']

# for col_name in target_columns:
#     dataframes = biu.create_total_column_by_units(dataframes, col_name, verbose=True)

In [11]:
# dataframes = biu.create_total_column_by_units(dataframes, 'Active Energy', verbose=True)

In [12]:
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['DataFrame Name', 'Time', '01 General_Electric_Active Energy (kWh)', '02 Production_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Energy (kWh)', '05 Compressors_Electric_Active Energy (kWh)', '06 Offices_Electric_Active Energy (kWh)', '07 Data Center_Electric_Active Energy (kWh)', '08 Technological Centers_Electric_Active Energy (kWh)', 'General_Electric_Active Energy (kWh)', '01 General_Electric_Power Factor (real)', '02 Production_Electric_Power Factor (real)', '03 Chiller Group_Electric_Power Factor (real)', '04 UTA_Electric_Power Factor (real)', '05 Compressors_Electric_Power Factor (real)', '06 Offices_Electric_Power Factor (real)', '07 Data Center_Electric_Power Factor (real)', '08 Technological Centers_Electric_Power Factor (real)', 'General_Electric_Power Factor (real)', '01 General_Electric_Active Power (kW)', '02 Production_Electric_Active Power (kW)', '03 Chiller Group_Electric_Ac

# Data Cleaning

## Imputing missing values with column averages by each dataframe

In [13]:
dataframes = du.impute_missing_values(dataframes)

DataFrame: industrial_site2 - NaNs filled with column means
DataFrame: industrial_site3 - NaNs filled with column means
DataFrame: industrial_site1 - NaNs filled with column means


In [14]:
du.nan_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time,day_of_week_cos,day_of_week_sin,is_weekend,month_cos,month_sin
0,No NaNs,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
1,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
2,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs


# Ingest Price Data

In [15]:
pd.DataFrame(du.get_min_max_dates(dataframes))

,industrial_site2,industrial_site3,industrial_site1
earliest_date,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00
latest_date,2023-09-19 09:45:00,2023-09-19 09:45:00,2023-09-19 09:45:00


In [16]:
import os
import re
from datetime import datetime, timedelta

# Define the directory containing the files
directory = 'data/energy_price_data/marginalpdbc'

# Regex pattern to match the filenames and extract dates
pattern = re.compile(r'marginalpdbc_(\d{8})\.\d')

# Get a list of files in the directory
files = os.listdir(directory)

# Extract dates from filenames
dates = []
for file in files:
    match = pattern.match(file)
    if match:
        date_str = match.group(1)
        date = datetime.strptime(date_str, '%Y%m%d')
        dates.append(date)

# Sort dates
dates.sort()

# Find the range of dates
if dates:
    start_date = dates[0]
    end_date = dates[-1]

    # Generate the complete date range
    complete_range = set(start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1))

    # Find missing dates
    missing_dates = complete_range - set(dates)

    # Print missing dates
    if missing_dates:
        print("Missing dates:")
        for missing_date in sorted(missing_dates):
            print(missing_date.strftime('%Y%m%d'))
    else:
        print("No missing dates.")
else:
    print("No valid files found in the directory.")

No missing dates.


In [17]:
csv_data = pd.read_csv('data/energy_price_data/spain_daily_market_hourly_prices/marginalpdbc_2022.csv')
csv_data.head()

,Date,Price1,Price2,Year
0,2022-03-18 00:00:00,203.00,203.00,2022
1,2022-03-18 01:00:00,199.00,199.00,2022
2,2022-03-18 02:00:00,198.88,198.88,2022
3,2022-03-18 03:00:00,198.88,198.88,2022
4,2022-03-18 04:00:00,198.88,198.88,2022


In [18]:
dataframes = epu.merge_price_data(dataframes, 'data/energy_price_data/spain_daily_market_hourly_prices/', 'Time', 'Date', 'Price1')


In [23]:
dataframes['industrial_site1']

,DataFrame Name,Time,01 Normal General_Electric_Active Energy (kWh),02 Chiller_Electric_Active Energy (kWh),03 Dyeing_Electric_Active Energy (kWh),04 Ironing_Electric_Active Energy (kWh),05 Purifier_Electric_Active Energy (kWh),07 Technological Centers_Electric_Active Energy (kWh),08 Offices Changing Rooms_Electric_Active Energy (kWh),09 Compressed Air_Electric_Active Energy (kWh),...,General_Electric_Active Power (kW),General_Natural Gas_Flow Rate (m3/h),General_Vapour_Flow Rate (m3/h),General_Water_Flow Rate (m3/h),is_weekend,month_sin,month_cos,day_of_week_sin,day_of_week_cos,Price1
0,industrial_site1,2022-07-01 00:00:00,441.00000,60.500000,60.400000,47.400000,27.900000,32.700000,14.600000,37.600000,...,471.000000,253.000000,8813.000000,766.000000,0,-0.5,-8.660254e-01,-0.433884,-0.900969,147.64
1,industrial_site1,2022-07-01 00:15:00,412.00000,60.700000,57.200000,46.200000,22.500000,27.700000,17.800000,40.700000,...,480.000000,0.090000,8636.000000,404.000000,0,-0.5,-8.660254e-01,-0.433884,-0.900969,147.64
2,industrial_site1,2022-07-01 00:30:00,440.00000,56.400000,62.400000,40.300000,24.000000,30.600000,17.700000,41.600000,...,519.000000,864.000000,16272.000000,784.000000,0,-0.5,-8.660254e-01,-0.433884,-0.900969,147.64
3,industrial_site1,2022-07-01 00:45:00,448.00000,54.400000,60.000000,48.600000,24.700000,30.700000,17.800000,40.100000,...,510.000000,275.000000,11009.000000,803.000000,0,-0.5,-8.660254e-01,-0.433884,-0.900969,147.64
4,industrial_site1,2022-07-01 01:00:00,410.00000,53.800000,59.500000,47.700000,24.700000,31.500000,17.500000,41.400000,...,520.000000,417.000000,10018.000000,665.000000,0,-0.5,-8.660254e-01,-0.433884,-0.900969,146.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42755,industrial_site1,2023-09-19 08:45:00,278.63457,19.070422,32.184718,42.741017,36.003673,26.064619,13.368608,74.741681,...,339.058803,322.114376,8749.661776,419.561495,0,-1.0,-1.836970e-16,0.781831,0.623490,129.82
42756,industrial_site1,2023-09-19 09:00:00,278.63457,19.070422,32.184718,42.741017,36.003673,26.064619,13.368608,74.741681,...,339.058803,322.114376,8749.661776,419.561495,0,-1.0,-1.836970e-16,0.781831,0.623490,114.69
42757,industrial_site1,2023-09-19 09:15:00,278.63457,19.070422,32.184718,42.741017,36.003673,26.064619,13.368608,74.741681,...,339.058803,322.114376,8749.661776,419.561495,0,-1.0,-1.836970e-16,0.781831,0.623490,114.69
42758,industrial_site1,2023-09-19 09:30:00,278.63457,19.070422,32.184718,42.741017,36.003673,26.064619,13.368608,74.741681,...,339.058803,322.114376,8749.661776,419.561495,0,-1.0,-1.836970e-16,0.781831,0.623490,114.69


In [27]:
import pandas as pd

def check_if_total_active_energy(df, suspect_column, energy_columns, verbose=False):
    """
    Check if the suspect column represents the total active energy by comparing it with the sum of other energy columns.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing the columns.
    - suspect_column (str): The name of the suspect total active energy column.
    - energy_columns (list of str): A list of column names representing individual energy usage.
    - verbose (bool): If True, print detailed comparison results.

    Returns:
    - bool: True if the suspect column matches the sum of other energy columns, False otherwise.
    """

    # Check if the suspect column and all energy columns exist in the DataFrame
    missing_columns = [col for col in [suspect_column] + energy_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"Columns not found in the DataFrame: {missing_columns}")

    # Sum the individual energy columns
    df['Sum_Energy_Columns'] = df[energy_columns].sum(axis=1)

    # Compare the suspect column with the sum of other energy columns
    columns_match = df[suspect_column].equals(df['Sum_Energy_Columns'])

    if verbose:
        if columns_match:
            print(f"The suspect column '{suspect_column}' matches the sum of the energy columns: {energy_columns}.")
        else:
            print(f"The suspect column '{suspect_column}' does not match the sum of the energy columns: {energy_columns}.")
            # Provide additional info on mismatched rows
            mismatched_rows = df[df[suspect_column] != df['Sum_Energy_Columns']]
            print(f"Mismatched rows:\n{mismatched_rows[[suspect_column] + ['Sum_Energy_Columns']].head()}")

    # Drop the temporary sum column
    df.drop(columns=['Sum_Energy_Columns'], inplace=True)

    return columns_match

# Example usage:
df = dataframes['industrial_site2']
energy_columns = ['02 Production_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Energy (kWh)','05 Compressors_Electric_Active Energy (kWh)','06 Offices_Electric_Active Energy (kWh)', '07 Data Center_Electric_Active Energy (kWh)', '08 Technological Centers_Electric_Active Energy (kWh)', 'General_Electric_Active Energy (kWh)']  # Replace with actual column names
result = check_if_total_active_energy(df, '01 General_Electric_Active Energy (kWh)', energy_columns, verbose=True)
# print("Does the suspect column represent total active energy?", result)

The suspect column '01 General_Electric_Active Energy (kWh)' does not match the sum of the energy columns: ['02 Production_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Energy (kWh)', '05 Compressors_Electric_Active Energy (kWh)', '06 Offices_Electric_Active Energy (kWh)', '07 Data Center_Electric_Active Energy (kWh)', '08 Technological Centers_Electric_Active Energy (kWh)', 'General_Electric_Active Energy (kWh)'].
Mismatched rows:
   01 General_Electric_Active Energy (kWh)  Sum_Energy_Columns
0                                    133.0               109.0
1                                    131.0               110.0
2                                    129.0               109.0
3                                    123.0                98.0
4                                    128.0               107.0


In [24]:
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['DataFrame Name', 'Time', '01 General_Electric_Active Energy (kWh)', '02 Production_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Energy (kWh)', '05 Compressors_Electric_Active Energy (kWh)', '06 Offices_Electric_Active Energy (kWh)', '07 Data Center_Electric_Active Energy (kWh)', '08 Technological Centers_Electric_Active Energy (kWh)', 'General_Electric_Active Energy (kWh)', '01 General_Electric_Power Factor (real)', '02 Production_Electric_Power Factor (real)', '03 Chiller Group_Electric_Power Factor (real)', '04 UTA_Electric_Power Factor (real)', '05 Compressors_Electric_Power Factor (real)', '06 Offices_Electric_Power Factor (real)', '07 Data Center_Electric_Power Factor (real)', '08 Technological Centers_Electric_Power Factor (real)', 'General_Electric_Power Factor (real)', '01 General_Electric_Active Power (kW)', '02 Production_Electric_Active Power (kW)', '03 Chiller Group_Electric_Ac

# Save data to CSVs

In [ ]:
du.save_dataframes_to_csv(dataframes, 'data/processed_data/industrial_sites')